--- 
Project Microeconometrics | Summer 2020, M.Sc. Economics, University of Bonn | [Vincent Selz](https://github.com/VincentSelz)

# Replication of “A Structural Analysis of Disappointment Aversion in a Real Effort Competition” by David Gill and Victoria Prowse (2012) <a class="tocSkip"> 
---

In [1]:
%matplotlib inline
from linearmodels.panel import RandomEffects
from linearmodels.panel import PanelOLS
import numpy as np
import pandas as pd
import pandas.io.formats.style
import respy as rp
import seaborn as sns
#from stargazer.stargazer import Stargazer
import statsmodels as sm
import statsmodels.formula.api as smf
import statsmodels.api as sm_api
import matplotlib as plt
from IPython.display import HTML
from IPython.display import display

from auxiliary.overview import *

/Users/vincentselz/opt/anaconda3/lib/python3.7/site-packages/respy/_numba.py:2: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.cgutils', please update to use 'numba.core.cgutils' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba import cgutils


In [4]:
pandas.set_option("display.precision", 3)

In [1]:
library(plm)
library(haven)
library(msm)
library(stargazer)

Warning message:
“package ‘plm’ was built under R version 3.5.2”
Warning message:
“package ‘msm’ was built under R version 3.5.2”

Please cite as: 


 Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.2. https://CRAN.R-project.org/package=stargazer 




---
# 1. Introduction 
---
Gill & Prowse (2012) use a real effort tournament in an laboratory experiment to test experimentally whether agents are disappointment averse. Pairs of subjects need to complete the real effort task sequentially. The payoff is determined in a lottery, where the probababilities of winning are linear in the difference in the agent's efforts. Hence, in the standard framework, the effort of the *First Mover* should not impact the choice of effort for the *Second Mover*. Yet, as predict by Gill & Prowse's model of disappointment aversion, subjects exhibit a discouragement effect, where high effort by the *First Mover*, impacts the effort by the *Second Mover* negatively. Their effort choices are strategic substitutes.

To estimate possible effects. First Gill & Prowse (2012) use a reduced form **random effects panel regression**. Given the substantial variation induced by their experimental design, they use the **Method of Simulated Moments (MSM)** to estimate the parameters of a structural model of disappointment aversion. This structural models allows them to estimate the strength of disappointment aversion on average but also to estimate the extent of heterogeneity in disappointment aversion across the population. 

This notebook will attempt to replicate the results of Gill & Prowse (2012) and will further add robustness to the findings by Gill & Prowse (2012). *Here could a preliminary result be*

Here, I walk the reader through the sections

---
# 2. Experimental Design
---
The six experimental session took place from late February to early March 2009 at the Nuffield Centre for Experimental Social Sciences (CESS) in Oxford. Per session twenty subjects participated at the study, leading to a subject pool of 120 participants in total. Each of the were paid a show-up fee of 4 pounds and went on to earn a further 10 pounds on average.  

## Slider Task 

The slider task consists of a single screen displaying numerous sliders. The number and position of sliders is not varied across subjects or across repetition of the task. At first, the sliders who range from 0 to 100 are placed at 0. It is possible to place the slider at every integer position between 0 and 100. The integer position is displaced next to the slider. The sliders can be adjusted and readjusted an unlimited number of times. The task for the subjects is to position as many sliders as possible at 50 during the time allocated to the subject. The number of sliders positioned at 50 constitutes the **points score**. The number of correctly placed sliders and the time remaining is displayed to the subject. In this experiment, 48 sliders were displayed and the subjects had 120 seconds to position them. 

**Potentially a picture with the task would help.**

## Experimental Procedure

Each session consists of 10 *First Mover* and 10 *Second Mover*, who stay in their role for the duration of their session. After two practice rounds, the subjects played 10 paying rounds. The prize for each pair was drawn randomly from {0.10,0.20,...3.90} and then revealed to the pair members. The task was completed sequentially, where the *Second Mover* observes the **points score** of the *First Mover*. This information is also displayed next to the alloted time and the own points score during the task. The probability to win the ensuing lottery is determined by

\begin{equation}
P_i(e_i,e_j) = \frac{e_i + e_j + 50}{100}
\end{equation}

were the $e_i$ in this setting corresponds to the **points score** of player $i$, and respectively $e_j$ for player $j$.

After each round the subjects were paired with another subjects. To ensure that subjects never meet each other, the “no contagion” matching algorithm of Cooper et al. (1996) is used. This prevents strategic action by the subject.  


---
# 3. Theoretical Predictions
---

---
# 4. Replication Random Effects Model
---

In [8]:
df_1, df_2, df_2_pref = get_datasets()
effort_overview(df_2_pref)

,Mean E1,SD E1,Mean E2,SD E2,Min E1,Min E2,Max E1,Max E2
period,,,,,,,,
1,22.034,5.991,21.763,6.101,1,0,33,34
2,22.627,6.708,23.458,4.836,0,11,33,33
3,24.763,6.075,24.831,4.875,0,12,37,38
4,24.627,5.956,25.203,4.502,0,16,35,36
5,24.966,6.800,25.119,5.660,0,0,36,35
6,24.729,7.508,24.898,7.039,1,0,37,39
7,25.881,5.855,25.763,6.109,9,0,37,37
8,26.831,5.858,26.169,5.133,9,14,41,35
9,25.593,8.550,26.254,6.702,0,0,38,40


In [10]:
exog_vars = ['e1','prize','e1timesprize','tt2','tt3','tt4', 'tt5','tt6','tt7','tt8','tt9','tt10']
result_2 = RandomEffects.from_formula('effort~1+e1+prize+e1timesprize+tt2+tt3+tt4+tt5+tt6+tt7+tt8+tt9+tt10',data=df_2).fit()
result_2

Dep. Variable:,effort,R-squared:,0.1366
Estimator:,RandomEffects,R-squared (Between):,-0.0108
No. Observations:,600,R-squared (Within):,0.1482
Date:,"Tue, Jun 30 2020",R-squared (Overall):,0.0436
Time:,18:48:15,Log-likelihood,-1644.8
Cov. Estimator:,Unadjusted,,
,,F-statistic:,7.7384
Entities:,60,P-value,0.0000
Avg Obs:,10.0000,Distribution:,"F(12,587)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,7.7384


In [7]:
result_1 = PanelOLS.from_formula('effort~1+e1+prize+e1timesprize+tt2+tt3+tt4+tt5+tt6+tt7+tt8+tt9+tt10+EntityEffects',data=df_2).fit()
result_1

Dep. Variable:,effort,R-squared:,0.1482
Estimator:,PanelOLS,R-squared (Between):,-0.0113
No. Observations:,600,R-squared (Within):,0.1482
Date:,"Thu, Jun 25 2020",R-squared (Overall):,0.0433
Time:,12:24:05,Log-likelihood,-1618.1
Cov. Estimator:,Unadjusted,,
,,F-statistic:,7.6546
Entities:,60,P-value,0.0000
Avg Obs:,10.0000,Distribution:,"F(12,528)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,7.6546


In [2]:
# import data 
df <- read_dta("data/paper_data/master_data.dta")
# convert the data frame to a data format recognizable by `plm` 
pdf = pdata.frame(df, index = c("Subject", "Period"))
pdf_pref = subset(pdf, !Subject==302)

In [3]:
re = plm(Effort~e1+prize+E1timesPrize+TT2+TT3+TT4+TT5+TT6+TT7+TT8+TT9+TT10, data = pdf, model = "random")
re_pref = plm(Effort~e1+prize+E1timesPrize+TT2+TT3+TT4+TT5+TT6+TT7+TT8+TT9+TT10, data = pdf_pref, model = "random")
fe = plm(Effort~e1+prize+E1timesPrize+TT2+TT3+TT4+TT5+TT6+TT7+TT8+TT9+TT10, data = pdf, model = "within")
fe_pref = plm(Effort~e1+prize+E1timesPrize+TT2+TT3+TT4+TT5+TT6+TT7+TT8+TT9+TT10, data = pdf_pref, model = "within")

In [4]:
phtest(fe,re)
phtest(fe_pref,re_pref)


	Hausman Test

data:  Effort ~ e1 + prize + E1timesPrize + TT2 + TT3 + TT4 + TT5 +  ...
chisq = 2.4266, df = 12, p-value = 0.9984
alternative hypothesis: one model is inconsistent



	Hausman Test

data:  Effort ~ e1 + prize + E1timesPrize + TT2 + TT3 + TT4 + TT5 +  ...
chisq = 2.6008, df = 12, p-value = 0.9978
alternative hypothesis: one model is inconsistent


In [6]:
stargazer(re_pref,re, type="text", title="Random Effects Regressions for Second Mover Effort",
          omit=c("TT2","TT3","TT4","TT5","TT6","TT7","TT8","TT9","TT10"),
          covariate.labels=c("First Mover Effort", "Prize", "Prize x First Mover effort","Intercept"),
          no.space=TRUE)
stargazer(fe_pref,fe, type="text", title="Fixed Effects Regressions for Second Mover Effort",
          omit=c("TT2","TT3","TT4","TT5","TT6","TT7","TT8","TT9","TT10"),
          covariate.labels=c("First Mover Effort", "Prize", "Prize x First Mover effort","Intercept"),
          no.space=TRUE)


Random Effects Regressions for Second Mover Effort
                               Dependent variable:     
                           ----------------------------
                                      Effort           
                                (1)            (2)     
-------------------------------------------------------
First Mover Effort             0.044          0.047    
                              (0.049)        (0.049)   
Prize                         1.639***      1.655***   
                              (0.602)        (0.592)   
Prize x First Mover effort    -0.049**      -0.050**   
                              (0.023)        (0.023)   
Intercept                    19.777***      19.392***  
                              (1.400)        (1.447)   
-------------------------------------------------------
Observations                    590            600     
R2                             0.137          0.135    
Adjusted R2                    0.119          0.117 

In [6]:
estmean <- coef(re)

estmean

(Intercept)           e1        prize E1timesPrize          TT2          TT3 
 19.39235270   0.04682713   1.65490418  -0.05017356   1.65466744   3.03435625 
         TT4          TT5          TT6          TT7          TT8          TT9 
  3.36900298   3.27873765   3.19325511   4.04271695   4.48732717   4.63204395 
        TT10 
  5.06565125

In [20]:
example(deltamethod)


dltmth> ## Simple linear regression, E(y) = alpha + beta x 
dltmth> x <- 1:100

dltmth> y <- rnorm(100, 4*x, 5)

dltmth> toy.lm <- lm(y ~ x)

dltmth> estmean <- coef(toy.lm)

dltmth> estvar <- summary(toy.lm)$cov.unscaled * summary(toy.lm)$sigma^2

dltmth> ## Estimate of (1 / (alphahat + betahat))
dltmth> 1 / (estmean[1] + estmean[2])
(Intercept) 
  0.2146277 

dltmth> ## Approximate standard error
dltmth> deltamethod (~ 1 / (x1 + x2), estmean, estvar) 
[1] 0.04328042

dltmth> ## We have a variable z we would like to use within the formula.
dltmth> z <- 1

dltmth> ## deltamethod (~ z / (x1 + x2), estmean, estvar) will not work.
dltmth> ## Instead, build up the formula as a string, and convert to a formula.
dltmth> form <- sprintf("~ %f / (x1 + x2)", z)

dltmth> form
[1] "~ 1.000000 / (x1 + x2)"

dltmth> deltamethod(as.formula(form), estmean, estvar)
[1] 0.04328042
